In [7]:
# step 01 : from erp (xls/html) to xlsx

import os
import pandas as pd
from bs4 import BeautifulSoup
import xlsxwriter

# Get all .xls files in the current directory
xls_files = [file for file in os.listdir() if file.endswith(".xls")]

# Loop over the .xls files and rename them to .html files
for xls_file in xls_files:
    # Get the new file name
    new_file_name = xls_file[:-4] + ".html"

    # Rename the file
    os.rename(xls_file, new_file_name)
	
# Get all HTML files in the current directory
html_files = [file for file in os.listdir() if file.endswith(".html")]

# Loop over the HTML files
for html_file in html_files:
    # Read the HTML file
    with open(html_file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    # Get the data in the HTML file
    data = soup.find_all("table")

    # Create a list of lists to store the data
    data_list = []
    for table in data:
        rows = table.find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            data_list.append([col.text for col in cols])

    # Create a Pandas DataFrame from the data
    df = pd.DataFrame(data_list)

    # Save the DataFrame to an Excel file
    df.to_excel(html_file[:-5] + ".xlsx", engine="xlsxwriter")
	
# Delete all HTML files in the current directory
for file in os.listdir():
    if file.endswith(".html"):
        os.remove(file)

In [8]:
# step 02 : remove 1 col and 12 rows

def remove_first_one_column_and_four_rows(file_path):
    """
    Remove the first 1 column and first 12 rows from an Excel file without saving the index row and number index.

    Args:
        file_path (str): The path to the Excel file.
    """

    df = pd.read_excel(file_path)
    df = df.iloc[13:, 1:]
    df = df.reset_index(drop=True)
    df.to_excel(file_path, index=False, header=None)


for file in os.listdir():
    if file.endswith(".xlsx"):
        file_path = os.path.join(os.getcwd(), file)
        remove_first_one_column_and_four_rows(file_path)

In [9]:
# step 03 : rename headers for all columns

import glob

# Get the list of Excel files in the current directory
excel_files = glob.glob('*.xlsx')  # Update the file extension if needed

# Assuming only one Excel file is present in the directory
if len(excel_files) == 1:
    file_name = excel_files[0]
    
    # Read the Excel file
    data = pd.read_excel(file_name)
    
    # Define the new column names
    new_column_names = [
        'trs_id', 'transaction_source', 'project_no', 'project_name', 'project_zone', 'task_no', 'task_name',
        'top_task_no', 'top_task_name', 'po_no', 'gl_date', 'expenditure_type', 'project_location',
        'project_floor', 'project_area', 'expenditure_category', 'expend_org', 'amount', 'line_no',
        'line_desc', 'inv_no', 'unit', 'qty', 'ipc_no', 'supplier_no', 'supplier_name', 'supplier_site',
        'comment', 'inventory_item', 'owner', 'distributions_status', 'distributions_date', 'distributions_details'
    ]

    # Rename the columns
    data.columns = new_column_names

    # Save the updated DataFrame back to the Excel file
    data.to_excel(file_name, index=False)
    print(f"Column headers in '{file_name}' have been renamed.")
else:
    print("There are zero or multiple Excel files in the directory. Please ensure there is exactly one Excel file.")


Column headers in '0169.RME_Projects_Cost_Distribution_181123.xlsx' have been renamed.


In [10]:
# Step 04 : fix number as text

import openpyxl as px

# Get a list of all Excel files in the current directory
excel_files = [file for file in os.listdir() if file.endswith('.xlsx')]

if not excel_files:
    print("No Excel files found in the current directory.")
else:
    # Select the first Excel file from the list
    file_name = excel_files[0]

    # Load the Excel file
    workbook = px.load_workbook(file_name)
    sheet = workbook.active

    # Iterate through the cells in column R (column 18) and remove commas, convert to numbers (amount)
    for row in sheet.iter_rows(min_row=2, min_col=18, max_col=18):
        for cell in row:
            if cell.value is not None and isinstance(cell.value, str):
                cell.value = cell.value.replace(',', '')
                try:
                    cell.value = float(cell.value)
                except ValueError:
                    pass

    # Iterate through the cells in column W (column 23) and remove commas, convert to numbers (qty)
    for row in sheet.iter_rows(min_row=2, min_col=23, max_col=23):
        for cell in row:
            if cell.value is not None and isinstance(cell.value, str):
                cell.value = cell.value.replace(',', '')
                try:
                    cell.value = float(cell.value)
                except ValueError:
                    pass

    # Save the modified Excel file with the same name as the old one
    workbook.save(file_name)

    print(f"Fixed Excel file saved as: {file_name}")

Fixed Excel file saved as: 0169.RME_Projects_Cost_Distribution_181123.xlsx


In [11]:
# step 05 : filter on marksup and create tab

# Find the Excel file in the current directory
excel_files = glob.glob("*.xlsx")

if not excel_files:
    print("No Excel file found in the directory.")
else:
    # Take the first Excel file found (you might adjust this logic if needed)
    file_name = excel_files[0]

    # Read the Excel file
    data = pd.read_excel(file_name)

    # Clean the 'expenditure_type' column by stripping whitespace
    data['expenditure_type'] = data['expenditure_type'].astype(str).str.strip()

    # Define the main criteria for filtering
    criteria = ["Bank Fees", "Stamp Duties", "Insurance", "Fees - stamp"]

    # Filter the data based on the cleaned 'expenditure_type' column
    filtered_data = data[data['expenditure_type'].isin(criteria)]

    # Create a new Excel file and write the filtered data to a new tab called 'marksup'
    with pd.ExcelWriter(file_name, engine='openpyxl', mode='a') as writer:
        filtered_data.to_excel(writer, sheet_name='marksup', index=False)

        # Calculate the sum of the 'amount' column for the filtered data
        total_amount = filtered_data['amount'].sum()

        # Write the total amount under the table of the filtered data in the 'marksup' tab
        workbook = writer.book
        worksheet = workbook['marksup']
        max_row = worksheet.max_row

        worksheet.cell(row=max_row + 2, column=1).value = "Total Amount"
        worksheet.cell(row=max_row + 2, column=2).value = total_amount

    # Save the changes using openpyxl
    book = load_workbook(file_name)
    book.save(file_name)




In [12]:
# step 06 : filter on penalties and create tab

from openpyxl import load_workbook

# Find the Excel file in the current directory
excel_files = glob.glob("*.xlsx")

if not excel_files:
    print("No Excel file found in the directory.")
else:
    # Take the first Excel file found (you might adjust this logic if needed)
    file_name = excel_files[0]

    # Read the Excel file
    data = pd.read_excel(file_name)

    # Clean the 'expenditure_type' column by stripping whitespace
    data['expenditure_type'] = data['expenditure_type'].astype(str).str.strip()

    # Define criteria for penalties filtering
    penalties_criteria = ["Penalty", "Penalty- Utilities", "Penalty- Row Material"]

    # Filter for 'penalties'
    penalties_data = data[data['expenditure_type'].isin(penalties_criteria)]

    # Create a new Excel file and write the penalties data to a new tab called 'penalties'
    with pd.ExcelWriter(file_name, engine='openpyxl', mode='a') as writer:
        penalties_data.to_excel(writer, sheet_name='penalties', index=False)

        # Calculate the sum of the 'amount' column for the penalties data
        total_penalties_amount = penalties_data['amount'].sum()

        # Write the total amount under the table of the penalties data in the 'penalties' tab
        workbook = writer.book
        penalties_worksheet = workbook['penalties']
        max_row_penalties = penalties_worksheet.max_row

        penalties_worksheet.cell(row=max_row_penalties + 2, column=1).value = "Total Penalties Amount"
        penalties_worksheet.cell(row=max_row_penalties + 2, column=2).value = total_penalties_amount

    # Save the changes using openpyxl
    book = load_workbook(file_name)
    book.save(file_name)



In [17]:
# step 07 : create pdf report of amount/marksup/penalties

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

# Find the Excel file in the current directory
current_directory = os.getcwd()
excel_files = [file for file in os.listdir(current_directory) if file.endswith(".xlsx")]

if not excel_files:
    print("No Excel file found in the directory.")
else:
    file_name = excel_files[0]

    # Load the Excel file containing 'marksup' and 'penalties' tabs
    book = load_workbook(file_name)

    # Get the first tab data
    first_tab_df = pd.read_excel(file_name, sheet_name=book.sheetnames[0])

    # Get the 'marksup' and 'penalties' data
    marks_df = pd.read_excel(file_name, sheet_name='marksup')
    penalties_df = pd.read_excel(file_name, sheet_name='penalties')

    # Calculate total original amount from the first tab
    original_amount = first_tab_df['amount'].sum()

    # Calculate total amount for 'marksup' and 'penalties'
    total_marksup = marks_df['amount'].sum()
    total_penalties = penalties_df['amount'].sum()

    # Calculate total direct and indirect amount
    total_direct_indirect = original_amount - total_marksup - total_penalties

    # Create a new worksheet for the report
    report_sheet = book.create_sheet('Report')

    # Write the report data to the worksheet with correct formatting
    report_sheet.append(['Category', 'Total Amount'])
    report_sheet.append(['Original Amount', f'{original_amount:.0f}'])
    report_sheet.append(['Total Marksup', f'{total_marksup:.0f}'])
    report_sheet.append(['Total Penalties', f'{total_penalties:.0f}'])
    report_sheet.append(['Total Direct and Indirect', f'{total_direct_indirect:.0f}'])

    # Apply a table style to the report
    tab = Table(displayName="ReportTable", ref="A1:B5")
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=True)
    tab.tableStyleInfo = style
    report_sheet.add_table(tab)

    # Save changes to the Excel file
    book.save(file_name)

    # Create a PDF from the report worksheet
    report_pdf_path = 'Report_Total_Amounts.pdf'
    report_canvas = canvas.Canvas(report_pdf_path, pagesize=letter)

    data = [
        ["Category", "Total Amount"],
        ["Original Amount", f'{original_amount:.0f}'],
        ["Total Marksup", f'{total_marksup:.0f}'],
        ["Total Penalties", f'{total_penalties:.0f}'],
        ["Total Direct and Indirect", f'{total_direct_indirect:.0f}']
    ]

    text_y = 750
    text_x = 100
    text_gap = 15

    text = report_canvas.beginText(text_x, text_y)
    text.setFont("Helvetica-Bold", 12)
    for row in data:
        text.textLine(f"{row[0]}: {row[1]}")
        text_y -= text_gap
    report_canvas.drawText(text)
    report_canvas.save()
